<a target="_blank" href="https://colab.research.google.com/github/wandb/eval-course/blob/main/notebooks/chapter_01_0.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<!--- @wandbcode{eval-course-01} -->

# Chapter 1: Introduction to LLM Evaluation

This code notebook is part of Chapter 1 lesson of the [LLM Apps: Evaluation course](https://wandb.ai/site/courses/evals/).

## Setup

In [1]:
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    !git clone --branch main https://github.com/wandb/eval-course
    %cd eval-course
    !pip install uv
    !uv pip install --system --quiet -r requirements.txt
    !uv pip install --system scipy==1.11.4
    %cd notebooks
else:
    print("Not running in Google Colab. Skipping git clone and pip install commands.")

Not running in Google Colab. Skipping git clone and pip install commands.


In [2]:
try:
    in_jupyter = True
except ImportError:
    in_jupyter = False
if in_jupyter:
    import nest_asyncio

    nest_asyncio.apply()

In [1]:
import asyncio
import json
import warnings
warnings.filterwarnings('ignore')

from dotenv import load_dotenv

load_dotenv()

import weave
from set_env import set_env

In [4]:
set_env("GOOGLE_API_KEY")
set_env("WANDB_API_KEY")
print("Env set")

2025-01-16 00:52:20.912 | WARNING  | set_env.set_env:set_env:109 - 
        Unable to set WANDB_API_KEY=WANDB_API_KEY,
        not in colab or Secrets not set, not kaggle
        or Secrets not set, no .env/dotenv/env file
        in the current working dir or parent dirs.


loading envfile='/Users/michaelliu/workspace/eval-course/eval-course/notebooks/.env' with dotenv_values(envfile)
loading envfile='/Users/michaelliu/workspace/eval-course/eval-course/notebooks/.env' with dotenv_values(envfile)
Env set


Set your W&B snd Google AI API keys as variables in the Colab secret tab.

In [5]:
from utils.config import ENTITY, MODEL, MODEL_CLIENT, WEAVE_PROJECT
from utils.evals import calculate_kappa_scores, get_evaluation_predictions
from utils.llm_client import LLMClient
from utils.prompts import (
    MedicalPrivacyJudgement,
    MedicalTaskScoreJudgement,
    medical_privacy_judge_prompt,
    medical_privacy_system_prompt,
    medical_system_prompt,
    medical_task,
    medical_task_score_prompt,
    medical_task_score_system_prompt,
)
from utils.render import display_prompt, print_dialogue_data
from utils.deserialize import MainCriteria, deserialize_model

## Understanding Medical Data Extraction Evaluation

### The Task: What Are We Trying to Do?

#### Raw Data Format
Medical conversations are messy and unstructured. Looking at our example data:

- Back-and-forth conversation between doctor and patient
- Contains personal details, small talk, and medical information mixed together
- Informal language ("hey", "mm-hmm", "yeah")
- Important details scattered throughout

#### Extraction Goals
The LLM needs to:
1. Find relevant information
2. Ignore irrelevant details
3. Standardize the format
4. Protect patient privacy
5. Maintain medical accuracy

In [6]:
if ENTITY is not None:
    weave_client = weave.init(f"{ENTITY}/{WEAVE_PROJECT}")
else:
    weave_client = weave.init(f"{WEAVE_PROJECT}")


Logged in as Weights & Biases user: michaelyliu6.
View Weave data at https://wandb.ai/michaelyliu6-none/eval_course_ch1/weave


In [7]:
display_prompt(medical_system_prompt)
display_prompt(medical_task)

You are a medical data extraction AI assistant. Your task is to accurately extract and summarize key medical       
information from patient records, adhering strictly to privacy guidelines and formatting instructions provided in  
the user's prompt. Focus on relevance and conciseness while ensuring all required fields are addressed.

You are extracting insights from medical records containing notes and doctor-patient dialogue.                     


                                                  Required Fields                                                  

Extract the following information:                                                                                 

• Chief complaint                                                                                                  

• History of present illness                                                                                       

• Physical examination                                                                                             

• Symptoms experienced by the patient                                                                              

• New medications prescribed or changed, including dosages (N/A if not provided)                                   

• Follow-up instructions (N/A if not provided)                                                                     


                                                   Requirements                                                    

• Do not include any personal identifiable information (PII)                                                       

• Use "the patient" instead of names                                                                               

• Format as bullet points: •field: value                                                                           

• Use N/A for missing values                                                                                       

• Keep response around 150 words                                                                                   

{transcript}

<div align="center">
    <img src="./media/medical_chatbot.png" width="250"/>
</div>

In [8]:
# Make sure to update the ENTITY and WEAVE_PROJECT in config.py to the correct project!
# Uncomment the following line to use your own annotated data after running chapter_01_generate_medical_data.ipynb
# annotated_medical_data = weave.ref(
#     f"weave:///{ENTITY}/{WEAVE_PROJECT}/object/medical_data_annotations:latest",
# ).get()
annotated_medical_data = weave.ref("weave:///a-sh0ts/eval_course_ch1_dev/object/medical_data_annotations:At9gri9UasftpPe5VNzT3EuIXQWAo5MYX8aMf2cuE8A").get()

In [9]:
print_dialogue_data(annotated_medical_data, indexes_to_show=[0], max_chars=500)

╭────────────────────────────────────────────────────  Input  ────────────────────────────────────────────────────╮
│                                                                                                                 │
│  input: Dialogue:  hey dylan what's going on so i lift quite a bit of weights i try to stay in shape as much    │
│  as i can i'm not like normal people i lift heavy weights and my elbow is extremely sore which elbow is it      │
│  actually it's both my elbows but my right elbow is hurting me the most  okay and you said you lift a lot of    │
│  weights  mm-hmm  did you play any sports when you were younger  no anything you can think of primarily it was  │
│  basketball baseb...                                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

### In fact, let's just generate an example now:

In [10]:
llm = LLMClient(model_name=MODEL, client_type=MODEL_CLIENT)
llm.predict(
    user_prompt=medical_task.format(transcript=annotated_medical_data[0][0]["input"]),
    system_prompt=medical_system_prompt,
)

🍩 https://wandb.ai/michaelyliu6-none/eval_course_ch1/r/call/01946dab-8932-7932-9036-eb4a686f2335


'• **Chief complaint**: Bilateral elbow pain, right worse than left.\n• **History of present illness**: The patient reports bilateral elbow pain for 1.5 years, worse on the right. The pain is located on the medial side of both elbows and is exacerbated by using their hands or arms. The patient lifts heavy weights for training. Ice does not help, the patient takes ibuprofen 800mg three times a day.\n• **Physical examination**: Limited range of motion on the right elbow extension and full range of motion with flexion, with pain in both. Pain on palpation of the medial side of the right elbow. Left elbow shows slight limited range of motion on extension with pain. Pain with supination in both elbows. Normal sensation in the fingers. Equal pulses in extremities.\n• **Symptoms**: Bilateral elbow pain, pain on flexion and extension of both elbows, pain with supination, limited range of motion with extension.\n• **New medications**: N/A\n• **Follow-up instructions**: The patient will get an M

## Data Collection and Curation for Evaluation

### Our approach for medical extraction evaluation data:

1. Start with real medical transcripts from production systems
   - Actual doctor-patient conversations 
   - Authentic medical terminology and flows
   - Real-world edge cases

2. Dataset Diversity Requirements
   - Various medical conditions
   - Different conversation styles
   - Mix of routine and complex cases
   - Remove duplicates for clean evaluation

In [11]:
print_dialogue_data(annotated_medical_data, indexes_to_show=[1], max_chars=2000)

╭───────────────────────────────────────────────────  Output  ────────────────────────────────────────────────────╮
│                                                                                                                 │
│  output:                                                                                                        │
│  • Chief complaint: Bilateral elbow pain, right worse than left                                                 │
│  • History of present illness: Patient, a 53-year-old male, has been experiencing bilateral elbow pain for      │
│  approximately 1.5 years, worse on the right side. Pain localized to the medial aspect, exacerbated by upper    │
│  extremity use. No relief with ice, taking ibuprofen 800 mg thrice daily. Actively uses heavy weights for       │
│  strength training and played basketball, baseball, and football in the past without pain.                      │
│  • Physical examination:                                                                                        │
│    - Right elbow: Limited extension range with pain, full flexion range with pain, medial aspect tenderness,    │
│  pain on supination.                                                                                            │
│    - Left elbow: Minimal pain with flexion and extension, limited extension range, pain on supination.          │
│  • Symptoms: Bilateral elbow pain, worse on right side, exacerbated by arm use.                                 │
│  • New medications: N/A                                                                                         │
│  • Follow-up instructions: MRI of bilateral elbows for further evaluation, possibility of whole blood           │
│  transfusion to encourage healing discussed.                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## Why and How to Evaluate LLMs

### Core Principles of LLM Evaluation
Unlike traditional software testing, LLM evaluation requires special consideration:

1. **Non-Deterministic Outputs**
   - Models can give different valid answers
   - Responses vary between runs
   - Multiple correct solutions possible

2. **Quality is Multi-Dimensional**
   - Correctness isn't binary
   - Context matters heavily
   - Different stakeholders have different priorities

3. **Scale vs Accuracy Trade-offs**
   - Manual review is accurate but expensive
   - Automated checks are scalable but limited
   - Hybrid approaches often work best

### Practical Evaluation Recipe 🧑‍🍳

1. **Define Success Criteria**
   - List must-have requirements
   - Set acceptable thresholds
   - Identify critical failures

2. **Build Evaluation Suite**
   - Automated checks for clear rules
   - Expert review for nuanced cases
   - Version control evaluation code

3. **Create Scoring System**
   - Establish baselines

### Applying to Medical Data Extraction 🏥

For our medical extraction task, this means:
- **Success Criteria**: Required fields, privacy compliance, word limits
- **Evaluation Suite**: Automated checks + medical expert review
- **Scoring**: Combination of format, accuracy, and safety metrics

Let's see how to implement this...

![](./media/traditional_llm_eval.png)

## Annotation: Building Quality Training Data

### Why Annotate?
To evaluate LLMs effectively, we need expert-labeled data that:
1. Defines what "good" looks like
2. Shows us what to test for
3. Helps align our automated tests with human judgment

### Ideal Process
Experts review outputs and provide structured feedback. This creates a foundation for:
- Building automated evaluation tests
- Measuring how well those tests match expert judgment
- Refining our evaluation methods until they align with expert standards

### Our Annotation Process
NOTE: In a production system, this would be done by licensed medical professionals using a strict rubric.
For this example code, we'll use synthetic annotations to demonstrate the process:

1. Binary Pass/Fail Judgments
   - Pass: Correctly extracted key medical information
   - Fail: Missed critical details or made dangerous assumptions

2. Detailed Critiques Required
   - For Passes: Document accuracy while noting improvement areas
   - For Fails: Identify specific medical extraction errors and their potential impact

These annotated examples become our evaluation dataset, though in practice, 
medical evaluations should always be validated by qualified healthcare professionals.

Think of annotations as our compass - they help ensure our later automated evaluation methods point in the same direction as human experts while assessing the quality of our LLM's outputs.

<div align="center">
    <img src="./media/annotation_ui.png" width="450"/>
</div>

In [12]:
print_dialogue_data(annotated_medical_data, indexes_to_show=[2, 3, 4], max_chars=500)

╭─────────────────────────────────────────────────  Annotation  ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  1                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────  Criteria Annotation  ─────────────────────────────────────────────╮
│                                                                                                                 │
│  MainCriteria(word_count=1, presence_of_keys=1, absence_of_PII=1)                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────────  Note  ─────────────────────────────────────────────────────╮
│                                                                                                                 │
│  The extraction successfully meets all specified criteria. All required fields are addressed, including chief   │
│  complaint, history of present illness, physical examination details, symptoms, new medications, and follow-up  │
│  instructions. The text is formatted correctly as a bullet list, with 'N/A' appropriately used for missing      │
│  medication information. No personal identifiable information is included, and the extraction remains concise,  │
│  adhering to the approximate 150-word limit. Overall, this is a we...                                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## Evaluation: Measuring Performance

### Understanding LLM Evaluation
Unlike traditional software testing, LLM evaluation requires multiple approaches:

1. **Automated Checks**
   - Fast, programmatic tests
   - Clear pass/fail criteria
   - Example: format rules, required fields

2. **Model-Assisted Evaluation**
   - Using LLMs to evaluate outputs
   - Helpful for subjective criteria
   - Example: checking medical accuracy, privacy compliance

3. **Expert Review**
   - Human validation of complex cases
   - Ground truth for training evaluators
   - Example: annotated datasets

### Building Evaluation Systems

In this notebook, we'll implement this through:

1. **Basic Tests**
   ```python
   test_adheres_to_required_keys()
   test_adheres_to_word_limit()
   ```

2. **LLM Judges**
   ```python
   judge_adheres_to_privacy_guidelines()
   judge_overall_score()
   ```

3. **Key Questions**
   - How closely do automated evaluations match human judgment?
   - When do automated systems diverge from human experts?
   - What makes a good evaluation system?

These questions lead us to the concept of alignment - measuring how well our automated systems match human expectations and values. We'll explore practical ways to measure and improve this alignment after implementing our evaluation system.

![](./media/eval_task_flowchart.png)

### Using Domain Knowledge to Build Evaluation Tests

We'll create four key tests to evaluate our medical extraction outputs:

1. **Required Fields Check**
   - Verifies presence of essential medical fields
   - E.g., "Chief complaint", "Symptoms", "Follow-up instructions"

2. **Word Limit Check**
   - Ensures output stays within 150-word limit
   - Promotes concise, focused summaries

3. **Privacy Guidelines Check**
   - Uses LLM to detect any PII leakage
   - Critical for medical data compliance

4. **Overall Quality Score**
   - LLM-based assessment of extraction quality
   - Considers accuracy, completeness, and format

These tests will be validated against our expert-annotated dataset to ensure they align with human judgment. This alignment process helps us understand how well our automated evaluation matches medical expert standards.

Let's implement each test:

#### Software Tests: Older and more rigid approach

In [13]:
test_output = annotated_medical_data[0][1]["output"]

In [14]:
@weave.op
def test_adheres_to_required_keys(model_output: str):
    # Required medical keys
    required_keys = [
        "Chief complaint",
        "History of present illness",
        "Physical examination",
        "Symptoms",
        "New medications with dosages",
        "Follow-up instructions",
    ]

    # Convert to lowercase for case-insensitive matching
    output_lower = model_output.lower()

    # Check if all required keys are present
    for key in required_keys:
        if key.lower() not in output_lower:
            return int(False)

    return int(True)

In [15]:
test_adheres_to_required_keys(test_output)

🍩 https://wandb.ai/michaelyliu6-none/eval_course_ch1/r/call/01946dab-9269-78a3-a63d-ff9bdb55ec08


0

In [16]:
@weave.op
def test_adheres_to_word_limit(model_output: str):
    return int(len(model_output.split()) <= 150)

In [17]:
test_adheres_to_word_limit(test_output)

🍩 https://wandb.ai/michaelyliu6-none/eval_course_ch1/r/call/01946dab-9270-77b1-a218-7d0e00b19a56


1

#### LLM Judges: Newer and more flexible approach

In [18]:
display_prompt(medical_privacy_system_prompt)
display_prompt(medical_privacy_judge_prompt)

You are a privacy compliance auditor specialized in medical records. Your task is to evaluate if any Personal      
Identifiable Information (PII) is present in the text.

Check for the following PII elements:                                                                              

• Names (patient, doctor, family members)                                                                          

• Dates of birth                                                                                                   

• Ages (if specific)                                                                                               

• Addresses                                                                                                        

• Phone numbers                                                                                                    

• Email addresses                                                                                                  

• Social Security numbers                                                                                          

• Medical record numbers                                                                                           

• Insurance information                                                                                            

• Specific dates of visits/procedures                                                                              

• Unique identifying characteristics                                                                               

• Geographic identifiers smaller than a state                                                                      

Return only two fields matching the following structure:                                                           

                                                                                                                   
 {{                                                                                                                
     "contains_pii": True/False,                                                                                   
     "reason": "Brief explanation of why PII was found or confirmation of privacy compliance"                      
 }}                                                                                                                
                                                                                                                   

Analyze this text for PII: {text}

In [19]:
@weave.op
def judge_adheres_to_privacy_guidelines(model_output: str):
    llm = LLMClient(model_name=MODEL, client_type=MODEL_CLIENT)
    response = llm.predict(
        user_prompt=medical_privacy_judge_prompt.format(text=model_output),
        system_prompt=medical_privacy_system_prompt,
        schema=MedicalPrivacyJudgement,
    )
    try:
        result = json.loads(response.text.strip("\n"))
        return int(not result[0]["contains_pii"])
    except:
        return int(True)

In [20]:
judge_adheres_to_privacy_guidelines(test_output)

🍩 https://wandb.ai/michaelyliu6-none/eval_course_ch1/r/call/01946dab-9280-7022-bc3d-cef374c0262a


1

In [21]:
display_prompt(medical_task_score_system_prompt)
display_prompt(medical_task_score_prompt)

You are a medical documentation quality assessor specialized in evaluating information extraction from medical     
records. Your task is to provide a single comprehensive score that is either True or False for the extracted       
information.

Scoring Criteria:                                                                                                  

 1 Required Fields (Critical)                                                                                      
   • All specified fields are present (chief complaint, history, examination, symptoms, medications, follow-up)    
   • Information is relevant and properly categorized                                                              
   • N/A is used appropriately for missing information                                                             
 2 Accuracy & Clarity                                                                                              
   • Information accurately reflects the source material                                                           
   • Medical terminology is used correctly                                                                         
   • Summaries are clear and unambiguous                                                                           
   • Key medical details are preserved                                                                             
 3 Privacy & Formatting                                                                                            
   • No personal identifiable information (PII) is included                                                        
   • "The patient" is used instead of names                                                                        
   • Bullet point format is followed correctly                                                                     
   • Response length is appropriate (~150 words)                                                                   
   • Information is well-organized and readable                                                                    

Return only two fields matching the following structure where the score MUST be either True or False:              

                                                                                                                   
 {{                                                                                                                
     "score": True/False,                                                                                          
     "reason": "Detailed explanation of why this score was given, referencing specific strengths and weaknesses"   
 }}                                                                                                                
                                                                                                                   

Evaluate this extraction: {text}

In [22]:
@weave.op
def judge_overall_score(model_output: str):
    llm = LLMClient(model_name=MODEL, client_type=MODEL_CLIENT)
    response = llm.predict(
        user_prompt=medical_task_score_prompt.format(text=model_output),
        system_prompt=medical_task_score_system_prompt,
        schema=MedicalTaskScoreJudgement,
    )
    try:
        result = json.loads(response.text.strip("\n"))
        return int(result[0]["score"])
    except:
        return int(False)

In [23]:
judge_overall_score(test_output)

🍩 https://wandb.ai/michaelyliu6-none/eval_course_ch1/r/call/01946dab-965d-7c31-8a7e-84a268bb689b


0

### We already have a dataset of annotated medical data. We can use our tests to evaluate the outputs of our LLM.

In [31]:
@weave.op
def annotated_data_passthrough(input, output):
    return output

In [25]:
evaluation_data = [
    {
        "input": annotated_row[0]["input"],
        "output": annotated_row[1]["output"],
        "scores": {
            "human_required_keys": deserialized_row.presence_of_keys,
            "human_word_limit": deserialized_row.word_count,
            "human_absence_of_PII": deserialized_row.absence_of_PII,
            "human_overall_score": annotated_row[2],
        },
    }
    for annotated_row in annotated_medical_data
    if (deserialized_row := deserialize_model(annotated_row[3], MainCriteria))
][0:5]

In [32]:
# Create evaluation
evaluation = weave.Evaluation(
    dataset=evaluation_data,
    scorers=[
        test_adheres_to_required_keys,
        test_adheres_to_word_limit,
        judge_adheres_to_privacy_guidelines,
        judge_overall_score,
    ],
)

# Run evaluation
evals = asyncio.run(evaluation.evaluate(annotated_data_passthrough))

Evaluated 1 of 5 examples

Evaluated 2 of 5 examples

Evaluated 3 of 5 examples

Evaluated 4 of 5 examples

Evaluated 5 of 5 examples

Evaluation summary
{
    'test_adheres_to_required_keys': {'mean': 0.0},
    'test_adheres_to_word_limit': {'mean': 1.0},
    'judge_adheres_to_privacy_guidelines': {'mean': 1.0},
    'judge_overall_score': {'mean': 0.0},
    'model_latency': {'mean': 0.002484893798828125}
}

🍩 https://wandb.ai/michaelyliu6-none/eval_course_ch1/r/call/01946db0-ee60-7801-936a-d4180d5c890b


### But do our test outputs adhere to the annotation expectations?

We need to measure how well our automated evaluations match human judgment. We'll:

1. **Measure Alignment**
   - Compare automated test results with expert annotations using kappa scores
   - Weight different aspects based on their importance (privacy, completeness, etc.)
   - Find where automated tests disagree with human experts

2. **Use These Results**
   - Chapter 2 will focus on improving the LLM judges that show poor alignment
   - We'll learn to refine prompts based on these alignment scores
   - Build better evaluation systems by focusing on the weakest areas first

These alignment measurements are crucial - they tell us which parts of our automated system need the most work, especially for critical aspects like privacy checks and medical accuracy.

In [27]:
# Get the evaluation call id from the evaluation object which you can see in the URL above!
# This line will break for you!
eval_call_id = "019468be-ab64-7a32-b8af-95861cd58715"

In [28]:
df = get_evaluation_predictions(weave_client, eval_call_id)
df

,input,required_keys,word_limit,privacy,overall
0,Dialogue: [doctor] hey dylan what's going on s...,"(1, 0)","(1, 1)","(1, 1)","(1, 0)"
1,"Dialogue: [doctor] hello , mrs . peterson . [p...","(0, 0)","(1, 1)","(1, 1)","(0, 0)"
2,"Dialogue: [doctor] hey , ms. hill . nice to se...","(1, 0)","(1, 1)","(1, 1)","(1, 0)"
3,"Dialogue: [doctor] hi keith , how are you ? [p...","(0, 0)","(1, 1)","(1, 1)","(0, 0)"
4,Dialogue: [doctor] okay so we are recording ok...,"(1, 0)","(1, 1)","(1, 1)","(1, 0)"


#### Software Tests: Minimal Alignment and hard to optimize

In [29]:
kappa_scores = calculate_kappa_scores(df, tuple_columns=["required_keys", "word_limit"])
for metric, score in kappa_scores.items():
    print(f"{metric}: {score:.3f}")

required_keys: 0.000
word_limit: nan


#### LLM Judges: Higher Alignment and easier to optimize

In [30]:
kappa_scores = calculate_kappa_scores(df, tuple_columns=["privacy", "overall"])
for metric, score in kappa_scores.items():
    print(f"{metric}: {score:.3f}")

privacy: nan
overall: 0.000


## Resources

- [Hamel's LLM Judge](https://hamel.dev/blog/posts/llm-judge/)
- [Hamel's LLM Evaluation](https://hamel.dev/blog/posts/evals/)
- [Clef's LLM Evaluation](https://huggingface.co/blog/clefourrier/llm-evaluation)
- [Eugene Yan's LLM Evaluators](https://eugeneyan.com/writing/llm-evaluators/)
- [Shreya's AI Engineering Flywheel](https://www.sh-reya.com/blog/ai-engineering-flywheel/)
- [Who Validates the Validators?](https://arxiv.org/abs/2404.12272)